# Getting Started
This tutorial demonstrates the configuration and use of a simple BSK-RL environment.
BSK-RL and dependencies should already be installed at this point (see [Installation](../install.rst)
if you haven't installed the package yet).

## Load Modules
In this tutorial, the environment will be created with `gym.make`, so it is necessary to
import the top-level `bsk_rl` module as well as `gym` and `bsk_rl` components.

In [ ]:
import gymnasium as gym
import numpy as np
from bsk_rl import act, data, obs, scene, sats
from bsk_rl.sim import dyn, fsw

from Basilisk.architecture import bskLogging
bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)


If no errors were raised, you have a functional installation of `bsk_rl`.

## Configure the Satellite
[Satellites](../api_reference/sats/index.rst) are configurable agents in the environment.
To make a new environment, start by specifying the [observations](../api_reference/obs/index.rst)
and [actions](../api_reference/act/index.rst) of a satellite type, as well as the underlying
Basilisk [simulation](../api_reference/sim/index.rst) models used by the satellite.

In [ ]:
class MyScanningSatellite(sats.AccessSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="storage_level_fraction"),
            dict(prop="battery_charge_fraction")
        ),
        obs.Eclipse(),
    ]
    action_spec = [
        act.Scan(duration=60.0),  # Scan for 1 minute
        act.Charge(duration=600.0),  # Charge for 10 minutes
    ]
    dyn_type = dyn.ContinuousImagingDynModel
    fsw_type = fsw.ContinuousImagingFSWModel

Based on this class specification, a list of configurable parameters for the satellite
can be generated.

In [ ]:
MyScanningSatellite.default_sat_args()

When instantiating a satellite, these parameters can be overriden with a constant or 
rerandomized every time the environment is reset using the ``sat_args`` dictionary.

In [ ]:
sat_args = {}

# Set some parameters as constants
sat_args["imageAttErrorRequirement"] = 0.05
sat_args["dataStorageCapacity"] = 1e10
sat_args["instrumentBaudRate"] = 1e7
sat_args["storedCharge_Init"] = 50000.0

# Randomize the initial storage level on every reset
sat_args["storageInit"] = lambda: np.random.uniform(0.25, 0.75) * 1e10

# Make the satellite
sat = MyScanningSatellite(name="EO1", sat_args=sat_args)

## Making the Environment
For this example, we will be using the single-agent [SatelliteTasking](../api_reference/index.rst) 
environment. Along with passing the satellite that we configured, the environment takes
a [scenario](../api_reference/scene/index.rst), which defines the environment the
satellite is acting in, and a [rewarder](../api_reference/data/index.rst), which defines
how data collected from the scenario is rewarded.

In [ ]:
env = gym.make(
    "SatelliteTasking-v1",
    satellite=sat,
    scenario=scene.UniformNadirScanning(),
    rewarder=data.ScanningTimeReward(),
    time_limit=5700.0,  # approximately 1 orbit
    log_level="INFO",
)

## Interacting with the Environment

First, the environment is reset.

In [ ]:
observation, info = env.reset(seed=1)

Next, we take the scan action (`action=0`) a few times. This allows for the satellite to
settle its attitude in the nadir pointing mode to satisfy imaging conditions. Note that 
the logs show little or no data accumulated in the first two steps as it settles, but
achieves 60 reward (corresponding to 60 seconds of imaging) by the third step.

In [ ]:
print("Initial data level:", observation[0], "(randomized by sat_args)")
for _ in range(3):
    observation, reward, terminated, truncated, info = env.step(action=0)
print("  Final data level:", observation[0])

The observation reflects the increase in stored data. The first element, corresponding
to `storage_level_fraction`, starts at a random value set by the `storageInit` function
in `sat_args` and increases based on the time spent imaging.

Finally, the charging mode is tasked repeatedly in 10-minute increments until the
environment time limit is reached.

In [ ]:
while not truncated:
    observation, reward, terminated, truncated, info = env.step(action=1)
    print(f"Charge level: {observation[1]:.3f} ({env.unwrapped.simulator.sim_time:.1f} seconds)\n\tEclipse: start: {observation[2]:.1f} end: {observation[3]:.1f}")

It is observed that the battery decrease while the satellite is in eclipse, but once the
satellite is out of eclipse, the battery quickly increases to full charge.